# Configuration

In [1]:
# Configuration
VOCAB_SIZE = 2000  # Size of the vocabulary
CONTEXT_LENGTH = 256  # Fixed context length for chunks
BATCH_SIZE = 256  # Batch size for training
TOKENIZER_FILE = "./data/tinystories-tokenizer"
CHUNK_FILE = "./data/chunked_stories"
SAMPLE_LIMIT = None  # Set to None to process the entire dataset
DICT_LABEL = 'seq'

In [2]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
import mlx.data as dx

from datasets import load_dataset

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

from llm.modules import SmallLanguageModel, loss_fn, create_causal_mask_triu
from llm.data import chunk_story, data_to_array_of_dict

# Model

In [3]:
model = SmallLanguageModel(vocab_dim=VOCAB_SIZE, embed_dim=4, n_head=2, num_layers=1)
x = mx.random.uniform(high=VOCAB_SIZE, shape=(32, 4)).astype(mx.int32)
# create mask to prevent attention to future tokens
mask = create_causal_mask_triu(x.shape[1])
output = model(x, mask)  # Forward pass

# Merge dataset to txt file

In [4]:
# from datasets import load_dataset
# ds = load_dataset("roneneldan/TinyStories")
# ds

In [5]:
# train_data = ds['train']
# print(f'Train data shape: {train_data.shape}')
# valid_data = ds['validation']
# print(f'Validation data shape: {valid_data.shape}')
# print('\n---------------------------------\n')
# print('This is a sample from the training data:\n')
# print(train_data[0]['text'])

In [6]:
# text_file_path = './data/tinystories_data.txt'

# with open(text_file_path, "w", encoding="utf-8") as f:
#     for example in train_data:
#         f.write(example['text'] + "\n")

# Tokenizer

In [7]:
if os.path.exists(f'{TOKENIZER_FILE}_{VOCAB_SIZE}.json'):
    print(f"Tokenizer file {TOKENIZER_FILE}_{VOCAB_SIZE}.json already exists. Skipping training.")
else:
    # Initialize a BPE tokenizer
    tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()

    # Configure the trainer with a vocabulary size and special tokens
    trainer = BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"])

    # Train the tokenizer on our text file
    print("Training tokenizer...")
    tokenizer.train(['./data/tinystories_data.txt'], trainer)
    print("Training complete.")

    # --- Save and Test the Tokenizer ---
    tokenizer_path = f'{TOKENIZER_FILE}_{VOCAB_SIZE}.json'
    tokenizer.save(tokenizer_path)
    print(f"Tokenizer saved to {tokenizer_path}")

    # Load it back and test
    tokenizer = Tokenizer.from_file(tokenizer_path)
    encoded = tokenizer.encode("Once upon a time, there was a little fox.")

    print("\n--- Testing the Tokenizer ---")
    print("Tokens:", encoded.tokens)
    print("IDs:", encoded.ids)

Tokenizer file ./data/tinystories-tokenizer_2000.json already exists. Skipping training.


# Chunking

In [8]:
if os.path.exists(f'{CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}.npz'):
    print(f"Chunk file {CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}.npz already exists. Skipping chunking.")
else:
    # Load the dataset (use a subset for testing)
    dataset = load_dataset("roneneldan/TinyStories", split="train")
    if SAMPLE_LIMIT:
        dataset = dataset.select(range(min(SAMPLE_LIMIT, len(dataset))))

    # Load the tokenizer
    tokenizer = Tokenizer.from_file(f'{TOKENIZER_FILE}_{VOCAB_SIZE}.json')

    # Process all stories and collect chunks
    all_chunks = []
    for story in tqdm(dataset["text"], desc="Chunking stories"):
        story_chunks = chunk_story(story, tokenizer, '[EOS]', '[PAD]', CONTEXT_LENGTH)
        all_chunks.extend(story_chunks)

    # Convert list to numpy array for efficient storage
    chunks_array = np.array(all_chunks, dtype=np.int32)

    # Print statistics
    print(f"Created {len(all_chunks)} chunks of length {CONTEXT_LENGTH}")
    print(f"Total tokens: {len(all_chunks) * CONTEXT_LENGTH:,}")
    print(f"Array shape: {chunks_array.shape}")

    # Save the chunks to a compressed file
    print(f"Saving chunks to {CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}.npz...")
    np.savez_compressed(f'{CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}.npz', chunks=chunks_array)
    print(f"Saved successfully! File size: {os.path.getsize(f'{CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}.npz') / (1024 * 1024):.2f} MB")

Chunk file ./data/chunked_stories_2000_256.npz already exists. Skipping chunking.


# Data Pipeline

In [9]:
data = np.load(f'{CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}.npz')
dicts = data_to_array_of_dict(data['chunks'], name=DICT_LABEL)

assert type(dicts) == list
assert type(dicts[0]) == dict
assert type(dicts[0][DICT_LABEL]) == np.ndarray

buffer = dx.buffer_from_vector(dicts)
stream = buffer.shuffle().to_stream().batch(32).prefetch(8,4)

In [10]:
for x in stream:
    print(x[DICT_LABEL].shape)
    print(type(x[DICT_LABEL]))
    break  # Just to test the first batch

(32, 256)
<class 'numpy.ndarray'>


# Training

In [12]:
tokenizer = Tokenizer.from_file(f'{TOKENIZER_FILE}_{VOCAB_SIZE}.json')
pad_token_id = tokenizer.token_to_id('[PAD]')
optimizer = optim.AdamW(learning_rate=0.001, betas=[0.9, 0.95], weight_decay=0.1)
loss_and_grad_fn = nn.value_and_grad(model, loss_fn)

In [14]:
for epoch in range(2):
    losses = []
    for i, seq in tqdm(enumerate(stream), desc="Processing batches"):
        mx_seq = mx.array(seq[DICT_LABEL])
        input_seq = mx_seq[:, :-1]  # Exclude the last token for input
        target_seq = mx_seq[:, 1:]  # Exclude the first token for target
        loss, grads = loss_and_grad_fn(model, input_seq, target_seq, pad_token_id)
        optimizer.update(model, grads)
        if i % 50 == 0:
            print("========================================================")
            print(f"Metal active memory: {mx.get_active_memory() / 1024**3:.2f} GB")
            print(f"Metal cache memory: {mx.get_cache_memory() / 1024**3:.2f} GB")
            print(f"Metal peak memory: {mx.get_peak_memory() / 1024**3:.2f} GB")
        # print(f"Batch {i + 1}, Loss: {loss:.4f}")
        # break
    # print average loss for the epoch with tqdm progress bar
    avg_loss = mx.array(losses).mean()
    tqdm.write(f"Epoch {epoch + 1}/{10}, Average Loss: {avg_loss:.4f}")

Processing batches: 0it [00:00, ?it/s]

Metal active memory: 0.00 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.00 GB
Metal active memory: 0.00 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.00 GB
Metal active memory: 0.00 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.00 GB
Metal active memory: 0.00 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.00 GB
Metal active memory: 0.01 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.01 GB
Metal active memory: 0.01 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.01 GB
Metal active memory: 0.01 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.01 GB
Metal active memory: 0.01 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.01 GB
Metal active memory: 0.01 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.01 GB
Metal active memory: 0.01 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.01 GB
Metal active memory: 0.02 GB
Metal cache memory: 0.00 GB
Metal peak memory: 0.02 GB
Metal active memory: 0.02 GB
Metal cache memory: 0.00 GB
Metal peak memory: 

RuntimeError: [metal::malloc] Resource limit (499000) exceeded.

In [ ]:
print(f'MLX current default device: {mx.default_device()}')

MLX current default device: Device(gpu, 0)
